In [1]:
import pandas as pd

filename = 'vo2max_data/vo2max_profile.xlsx'
profile = pd.read_excel(filename)
profile = profile[profile['name'].notna()]
profile.dropna(axis='columns', inplace=True)
profile['name'] = profile['name'].apply(lambda x: x.split(' ')[0].lower())
profile['gender'] = profile['gender'].astype(int)
print(profile)

    label      name  gender   age         ID  warm_up_speed
0     1.0     james       1  29.0  2791829.0            5.0
1     2.0       jun       1  36.0  2791820.0            5.0
2     3.0  mingchia       1  40.0  2791870.0            5.0
3     4.0     lance       1  34.0  2791818.0            6.0
4     5.0     metis       0  26.0  2791781.0            6.0
5     6.0      liam       1  33.0  2791845.0            8.0
6     7.0      show       1  27.0  2791784.0            6.0
7     8.0     terry       1  40.0  2791817.0            7.0
8     9.0      jing       1  37.0  2791789.0            6.0
9    10.0       may       0  33.0  2791787.0            6.0
10   11.0       ace       1  43.0  2791785.0            5.0
11   12.0     stone       1  36.0  2791814.0            6.5
12   13.0     frank       1  37.0  2791793.0            8.0
13   14.0      gary       1  46.0  2791792.0            6.0
14   15.0        yt       1  33.0  2791823.0            8.0
15   16.0      pink       0  37.0  27917

In [2]:
import pandas as pd
import os
import math
import csv

name_list = os.listdir('vo2max_data/process/')
name_list = [x.split('.')[0] for x in name_list]
min_time = 999999
for name in name_list:
    workout = pd.read_excel('vo2max_data/process/' + name + '.xlsx')
    firstname = name.split('_')[0]
    warm_up_speed = profile[profile['name'] == firstname]['warm_up_speed'].values[0]
    gender = profile[profile['name'] == firstname]['gender'].values[0]
    gender_mapping_speed = [6.0, 8.0]
    workout.drop(columns=['Marker', 'V\'O2', 'V\'O2/kg', 'V\'O2/HR'], inplace=True)
    workout = workout[workout['t'].notna()]
    workout = workout[workout['Phase'].isin(['Rest', 'Warm Up', 'Exercise'])]
    workout['t'] = workout['t'].apply(lambda x: int(x.split(':')[-1].split('.')[0]) + 60*int(x.split(':')[1]))
    workout_list = workout.values.tolist()
    hr_prev = 0.0
    ts_prev = 0
    pad_workout = []
    pad_workout.append([0, 'Rest', workout_list[0][2], 0])
    for i in range(len(workout_list)):
        ts_prev = pad_workout[-1][0]
        state_prev = pad_workout[-1][1]
        hr_prev = pad_workout[-1][2]
        ts_now = workout_list[i][0]
        hr_now = workout_list[i][2]
        # if ts_now > 900:
        #     break
        diff_ts = ts_now - ts_prev
        if diff_ts == 0:
            continue
        hr_step = (hr_now - hr_prev) / diff_ts
        for j in range(ts_prev+1, ts_now):
            if j < 300:
                speed = 0
            elif j <= 600 and j >= 300:
                speed = warm_up_speed
            else:
                speed = math.ceil(j/60 - 10) - 1 + gender_mapping_speed[gender]
            pad_workout.append([j, state_prev, hr_prev+hr_step*(j-ts_prev), speed])
        if ts_now < 300:
            speed = 0
        elif ts_now <= 600 and ts_now >= 300:
            speed = warm_up_speed
        else:
            speed = math.ceil(ts_now/60 - 10) - 1 + gender_mapping_speed[gender]
            
        new_record = workout_list[i].copy()
        new_record.append(speed)
        pad_workout.append(new_record)
    pad_workout = [record for record in pad_workout if record[1] != 'Rest']
    min_time = min(min_time, pad_workout[-1][0])
    with open('vo2max_data/full_csv/' + name + '.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['speed', 'hr'])
        for i in range(len(pad_workout)):
            writer.writerow([str(pad_workout[i][-1]), str(pad_workout[i][-2])])
    # print(pad_workout)
    # break


In [53]:
min_time

863